In [1]:
# Pull in Dependences
import pandas as pd
import numpy as np

### Load the Data

In [2]:
# Load Airbnb listings data from CSV file
airbnb_data = pd.read_csv('Resources/raw_airbnb_listings_usa.csv', dtype={'neighbourhood_group': str})

# Display the first few rows of the data
airbnb_data.head()


,Unnamed: 0,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,...,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,state,city
0,0,183319,Panoramic Ocean View Venice Beach,867995,Barbara X,City of Los Angeles,Venice,33.99211,-118.47600,Entire home/apt,...,30,3,2019-02-25,0.02,2,0,0,NaN,CA,Los Angeles
1,1,109,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.98301,-118.38607,Entire home/apt,...,30,2,2016-05-15,0.01,1,139,0,NaN,CA,Los Angeles
2,2,51307,Spanish Bungalow Guest House LA CA. 30 plus ni...,235568,David,City of Los Angeles,Atwater Village,34.12206,-118.26783,Entire home/apt,...,30,138,2020-12-13,0.98,2,224,0,NaN,CA,Los Angeles
3,3,184314,Boho Chic Flat..Steps to Beach!,884031,Ashley,City of Los Angeles,Venice,33.97487,-118.46312,Entire home/apt,...,30,30,2017-12-24,0.22,1,0,0,NaN,CA,Los Angeles
4,4,51498,Guest House With Its Own Entrance/Exit and Hot...,236758,Bay,City of Los Angeles,Mar Vista,34.00389,-118.44126,Entire home/apt,...,3,378,2022-08-21,2.60,1,348,41,HSR19-001336,CA,Los Angeles


### Inspect the Data

In [3]:
# Display data information
airbnb_data.info()

# Check for missing values
airbnb_data.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325858 entries, 0 to 325857
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      325858 non-null  int64  
 1   id                              325858 non-null  int64  
 2   name                            325839 non-null  object 
 3   host_id                         325858 non-null  int64  
 4   host_name                       324714 non-null  object 
 5   neighbourhood_group             155047 non-null  object 
 6   neighbourhood                   325146 non-null  object 
 7   latitude                        325858 non-null  float64
 8   longitude                       325858 non-null  float64
 9   room_type                       325858 non-null  object 
 10  price                           325858 non-null  int64  
 11  minimum_nights                  325858 non-null  int64  
 12  number_of_review

Unnamed: 0                             0
id                                     0
name                                  19
host_id                                0
host_name                           1144
neighbourhood_group               170811
neighbourhood                        712
latitude                               0
longitude                              0
room_type                              0
price                                  0
minimum_nights                         0
number_of_reviews                      0
last_review                        62692
reviews_per_month                  62692
calculated_host_listings_count         0
availability_365                       0
number_of_reviews_ltm                  0
license                           239223
state                                  0
city                                   0
dtype: int64

### Handling Missing Values

In [4]:
# Fill missing values in 'reviews_per_month' with 0
airbnb_data['reviews_per_month'].fillna(0, inplace=True)

# Drop rows with missing values in crucial columns (e.g., 'name', 'host_id')
airbnb_data.dropna(subset=['name', 'host_id'], inplace=True)

# Verify missing values handling
airbnb_data.isnull().sum()


C:\Users\Bryan Hamilton-Brown\AppData\Local\Temp\ipykernel_16216\3082303304.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airbnb_data['reviews_per_month'].fillna(0, inplace=True)


Unnamed: 0                             0
id                                     0
name                                   0
host_id                                0
host_name                           1144
neighbourhood_group               170809
neighbourhood                        712
latitude                               0
longitude                              0
room_type                              0
price                                  0
minimum_nights                         0
number_of_reviews                      0
last_review                        62680
reviews_per_month                      0
calculated_host_listings_count         0
availability_365                       0
number_of_reviews_ltm                  0
license                           239204
state                                  0
city                                   0
dtype: int64

### Format Consistences

In [5]:
# Ensure data types are consistent
airbnb_data['last_review'] = pd.to_datetime(airbnb_data['last_review'], errors='coerce')

# Check data types
airbnb_data.dtypes


Unnamed: 0                                 int64
id                                         int64
name                                      object
host_id                                    int64
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
number_of_reviews_ltm                      int64
license                                   object
state                                     object
city                

### Remove Duplicates

In [6]:
# Remove duplicate rows based on 'id' or 'host_id'
airbnb_data = airbnb_data.drop_duplicates(subset=['id'])

# Verify removal of duplicates
airbnb_data.duplicated().sum()

0

### Remove Irrelevant Columns

In [7]:
# Define the relevant columns for analysis
relevant_columns = [
    'id', 'name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
    'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews',
    'last_review', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365'
]

# Select only the relevant columns
airbnb_data = airbnb_data[relevant_columns]

# Verify the columns
airbnb_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 274773 entries, 0 to 325857
Data columns (total 15 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   id                              274773 non-null  int64         
 1   name                            274773 non-null  object        
 2   host_id                         274773 non-null  int64         
 3   neighbourhood_group             115930 non-null  object        
 4   neighbourhood                   274061 non-null  object        
 5   latitude                        274773 non-null  float64       
 6   longitude                       274773 non-null  float64       
 7   room_type                       274773 non-null  object        
 8   price                           274773 non-null  int64         
 9   minimum_nights                  274773 non-null  int64         
 10  number_of_reviews               274773 non-null  int64       

### Filter Data based on Number of Reviews

In [8]:
# Set the threshold for the minimum number of reviews
min_reviews_threshold = 100

# Filter the data to include only listings with at least 'min_reviews_threshold' reviews
filtered_airbnb_data = airbnb_data[airbnb_data['number_of_reviews'] >= min_reviews_threshold].copy()

# Verify the filtering
filtered_airbnb_data.info()

# Display the first few rows of the filtered data
filtered_airbnb_data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 33237 entries, 2 to 325827
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              33237 non-null  int64         
 1   name                            33237 non-null  object        
 2   host_id                         33237 non-null  int64         
 3   neighbourhood_group             11277 non-null  object        
 4   neighbourhood                   33147 non-null  object        
 5   latitude                        33237 non-null  float64       
 6   longitude                       33237 non-null  float64       
 7   room_type                       33237 non-null  object        
 8   price                           33237 non-null  int64         
 9   minimum_nights                  33237 non-null  int64         
 10  number_of_reviews               33237 non-null  int64         
 11  last_r

,id,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
2,51307,Spanish Bungalow Guest House LA CA. 30 plus ni...,235568,City of Los Angeles,Atwater Village,34.12206,-118.26783,Entire home/apt,75,30,138,2020-12-13,0.98,2,224
4,51498,Guest House With Its Own Entrance/Exit and Hot...,236758,City of Los Angeles,Mar Vista,34.00389,-118.44126,Entire home/apt,189,3,378,2022-08-21,2.60,1,348
9,51546,Cool Pad Under the Hollywood Sign,237114,City of Los Angeles,Hollywood Hills,34.11874,-118.32113,Entire home/apt,100,31,188,2022-04-29,1.30,1,250
11,187526,Malibu Pacific Ocean View Room 1,215462,Other Cities,Malibu,34.03073,-118.75513,Private room,350,2,196,2022-08-24,1.58,4,365
12,52871,Beverly Hills Large 1-Bdr Near Everything,245280,Other Cities,Beverly Hills,34.06118,-118.39804,Entire home/apt,165,5,130,2022-08-16,0.93,1,259


### Sort Data

In [9]:
# Sort the data based on 'number_of_reviews' in descending order
filtered_airbnb_data.sort_values(by='number_of_reviews', ascending=False, inplace=True)

# Display the first few rows of the sorted data
filtered_airbnb_data

,id,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
186785,29819757,Hotel Perks - Private Bedroom | Private Bathroom,57387860,NaN,Near North Side,41.89230,-87.62734,Hotel room,284,1,2600,2022-08-30,55.71,24,346
125702,8357,The Mushroom Dome Retreat & LAND of Paradise S...,24281,NaN,Unincorporated Areas,37.00939,-121.88547,Entire home/apt,159,2,1724,2022-09-25,10.73,2,100
17954,42409434,The Burlington Hotel,229716119,City of Los Angeles,Westlake,34.05981,-118.26963,Entire home/apt,113,1,1702,2022-09-06,56.17,5,124
222414,35158303,"SAHARA Las Vegas, Marra 1 King",263748930,NaN,Unincorporated Areas,36.14215,-115.15706,Private room,238,1,1542,2022-09-05,83.80,11,298
215898,44799007,Sonder Battery Park | Studio Apartment,219517861,NaN,Ward F (councilmember Jermaine D. Robinson),40.70617,-74.01486,Entire home/apt,521,2,1518,2022-09-03,62.13,31,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97415,19787549,UTC Townhouse 3 miles La Jolla Beach UCSD Scripts,3396026,NaN,University City,32.86786,-117.22282,Entire home/apt,316,2,100,2022-09-12,1.59,1,90
193069,13276296,Frenchmen Street Studio,148087676,NaN,Marigny,29.96678,-90.05787,Entire home/apt,98,3,100,2020-03-14,1.33,2,0
285845,8649816,Chill East Nashville House // Entire Home,34836262,NaN,District 6,36.18213,-86.70756,Entire home/apt,299,3,100,2022-09-11,1.32,1,308
44491,39753984,15th St - 2 Bedroom BACK Apt at OCEANFRONT HOME,47662934,NaN,Newport Beach,33.60655,-117.92164,Entire home/apt,348,3,100,2022-08-28,3.03,118,353


In [10]:
# Save the cleaned, filtered, and sorted data to a new CSV file
filtered_airbnb_data.to_csv('Resources/cleaned_airbnb_listings_usa.csv', index=False)
